In [1]:
from functools import reduce
import pandas as pd
from datetime import datetime

import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql import Window

In [2]:
spark = SparkSession.builder.getOrCreate()

21/12/14 15:55:53 WARN Utils: Your hostname, emif-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.18 instead (on interface en0)
21/12/14 15:55:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/14 15:55:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/12/14 15:55:55 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/12/14 15:55:55 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [10]:
# sessions = "1000,1"

def format_nrb(input_sdf: DataFrame, nbr_cols: list) -> DataFrame:

    df_out = reduce(
        lambda df, col: df.withColumn(
            col, F.regexp_replace(F.col(col), "[^0-9]", "").cast("int")
        ),
        nbr_cols,
        input_sdf,
    )
    return df_out

In [19]:

columns = ["language","users_count"]
data = [("Java", "1,234"), ("Python", "1,000,000"), ("Scala", "5,432"), ("Kzov", "2")]


In [20]:

df = spark.createDataFrame(data).toDF(*columns)


In [21]:
df.show()
df.printSchema()

+--------+-----------+
|language|users_count|
+--------+-----------+
|    Java|      1,234|
|  Python|  1,000,000|
|   Scala|      5,432|
|    Kzov|          2|
+--------+-----------+

root
 |-- language: string (nullable = true)
 |-- users_count: string (nullable = true)



In [22]:
cols_list = ["users_count",]

df_output = format_nrb(df, cols_list)

df_output.printSchema()
df_output.show()


root
 |-- language: string (nullable = true)
 |-- users_count: integer (nullable = true)

+--------+-----------+
|language|users_count|
+--------+-----------+
|    Java|       1234|
|  Python|    1000000|
|   Scala|       5432|
|    Kzov|          2|
+--------+-----------+

